This notebook is the beginning of scaling up First-Order-Motion-Model to use, and generate, larger scale images (e.g. 512x512px).

It takes video processing repositories and the First-Order-Motion-Model and begins the training process in PNG format inputs.

Lot's still to do :) <br>
TODO:
- Intialize training from a checkpoint if possible.
- Throw many GPUs at training and verify correct behavior.
- Refine architecture to something more modern (why are we not using transformers for all of their benifits with attention?)
- etc.

In [1]:
!nvidia-smi

/usr/bin/sh: 1: nvidia-smi: not found


In [2]:
import os
from pathlib import Path

base_path = Path.home()
print(base_path)

/home/jupyter


In [3]:
VIDEO_PREPROCESSING_REPO = f'{base_path}/video-preprocessing'
# !git clone https://github.com/jacobwjs/video-preprocessing $VIDEO_PREPROCESSING_REPO

%cd $VIDEO_PREPROCESSING_REPO

/home/jupyter/video-preprocessing


In [4]:
# !pip install -r requirements.txt

## Restart runtime to recognize/use newly installed packages.
- Continue in cell below once restarted. 

In [7]:
import pandas as pd 
import numpy as np
import os
from pathlib import Path

base_path = Path.home()
print(base_path)

VIDEO_PREPROCESSING_REPO = f'{base_path}/video-preprocessing'
# !git clone https://github.com/jacobwjs/video-preprocessing $VIDEO_PREPROCESSING_REPO
%cd $VIDEO_PREPROCESSING_REPO

Read in the original metadata file and select a (small) subset of the videos to test 512x512px inputs in PNG format.

In [11]:
df = pd.read_csv(f'{VIDEO_PREPROCESSING_REPO}/vox-metadata.csv')
display(df.iloc[0:5]) # Display a few entries.

,video_id,start,end,bbox,fps,width,height,partition,person_id
0,WMh3ye5tBsE,3657,3994,624-48-1018-553,25,1280,720,train,id10005
1,WMh3ye5tBsE,3467,3576,610-41-989-550,25,1280,720,train,id10005
2,WMh3ye5tBsE,1426,1546,522-49-931-580,25,1280,720,train,id10005
3,WMh3ye5tBsE,3292,3435,600-59-1095-540,25,1280,720,train,id10005
4,WMh3ye5tBsE,4022,4235,534-42-1021-572,25,1280,720,train,id10005


In [12]:
def get_bbox_dims(x):
    '''
    Calculate the dimensions of the bounding box (e.g. (512, 512)).
    Args:
        df['bbox']: coordinates of bbox.
    Returns:
        dims (tuple): dimensions of the bbox.
    '''
    debug = False # True

    left, top, right, bot = list(map(int, x.split('-')))
    dims = (abs(left - right), abs(top - bot))
    if debug: print(dims)
    
    return dims

Calculate the dimensions of the bounding box in order to include (or remove) certain videos from the training inputs.
- Not sure why this is necessary, as interpolation to 512px is happening regardless, but comes as advice from https://github.com/AliaksandrSiarohin/first-order-model/issues/81

In [14]:
# df['column']=df['column'].apply(get_bbox_dims)
df_copy = df.copy()
df_copy['bbox_dims'] = df_copy['bbox'].apply(get_bbox_dims)

In [16]:
dims = (512, 512)
# df_bbox_cutoff = df.loc[df['bbox_dims'] >= dims & df['partition'] == 'train'] # For only training data.
df_bbox_cutoff = df_copy.loc[df_copy['bbox_dims'] >= dims]
print("Found %d videos with bbox >= %s out of %d" % (len(df_bbox_cutoff), dims, len(df)))
# display(df_bbox_cutoff)

Found 1651 videos with bbox >= (512, 512) out of 20075


In [52]:
# from datetime import datetime 

# start_time = datetime.now() 

# j = 0
# for i in range(int(10e3)):
#     j += 1

# time_elapsed = datetime.now() - start_time 
# print('Time elapsed (hh:mm:ss.ms) {}'.format(time_elapsed))

Time elapsed (hh:mm:ss.ms) 0:00:00.000958


Process a small subset of the videos just for verification we can successfully kick off training.

In [24]:
# Save subset of videos to process.
# ------------------------------------------

# df_to_save = df_copy

# # Remove the new column, no longer needed, and not sure how parsing
# # routines in the repo will treat it.
# #
# df_to_save.drop('bbox_dims', axis=1, inplace=True) 

# # Only keep the first 3 entries. 
# # NOTE:
# # - Remove this to save them all.
# #
# # df_to_save.drop(range(25, len(df_to_save)), axis=0, inplace=True) # First 25 
# df_to_save = df_to_save.iloc[range(0, 5)]
# display(df_to_save)


# Save all videos to process.
# ------------------------------------------
df_to_save = df

# Remove the new column, no longer needed, and not sure how parsing
# routines in the repo will treat it.
#
try:
    df_to_save.drop('bbox_dims', axis=1, inplace=True) 
except:
    print("already removed bbox_dims...")

    
# Write to disk so we can use it with the script provided by the repo to 
# read in videos and crop.
#
# df_to_save = df_to_save.iloc[range(0,1)]
display(df_to_save)
df_to_save.to_csv(f'{VIDEO_PREPROCESSING_REPO}/vox-metadata-512px.csv', index=False)

already removed bbox_dims...


,video_id,start,end,bbox,fps,width,height,partition,person_id
0,WMh3ye5tBsE,3657,3994,624-48-1018-553,25,1280,720,train,id10005
1,WMh3ye5tBsE,3467,3576,610-41-989-550,25,1280,720,train,id10005
2,WMh3ye5tBsE,1426,1546,522-49-931-580,25,1280,720,train,id10005


Start the downloads based on the updated vox-metadata file.

In [23]:
!wget https://yt-dl.org/downloads/latest/youtube-dl -O youtube-dl
!chmod a+rx youtube-dl

--2020-11-28 05:46:57--  https://yt-dl.org/downloads/latest/youtube-dl
Resolving yt-dl.org (yt-dl.org)... 95.143.172.170, 2001:1a50:11:0:5f:8f:acaa:177
Connecting to yt-dl.org (yt-dl.org)|95.143.172.170|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://yt-dl.org/downloads/2020.11.26/youtube-dl [following]
--2020-11-28 05:46:58--  https://yt-dl.org/downloads/2020.11.26/youtube-dl
Connecting to yt-dl.org (yt-dl.org)|95.143.172.170|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/ytdl-org/youtube-dl/releases/download/2020.11.26/youtube-dl [following]
--2020-11-28 05:46:59--  https://github.com/ytdl-org/youtube-dl/releases/download/2020.11.26/youtube-dl
Resolving github.com (github.com)... 52.192.72.89
Connecting to github.com (github.com)|52.192.72.89|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github-production-release-asset-2e65be.s3.amazonaws.com/1039520/7769

## MP4 version

In [ ]:
# %cd $VIDEO_PROCESSING_REPO
# !python load_videos.py --metadata vox-metadata-512px.csv \
                        #  --format .mp4 \
                        #  --out_folder vox_mp4_512px 
                        #  --workers 4

In [ ]:
# !zip -r /content/vox_videos_512px.zip /content/video-preprocessing/vox_mp4_512px

In [ ]:
# from google.colab import files
# files.download("/content/vox_videos_slim.zip")

## PNG version

** Highly recommended to close jupyterlab and ssh into machine and run the following**

<code> python load_videos.py --metadata vox-metadata-512px.csv \
                        --format .png \
                        --image_shape "512,512" \
                        --out_folder vox_png_512px \
                        --workers 8"
</code>

In [39]:
# !rm -rf /content/video-processing/vox_png_512px
print("Saving pngs to: ", base_path)

Saving pngs to:  /home/jupyter


In [37]:
# !python load_videos.py --metadata vox-metadata-512px.csv \
#                         --format .png \
#                         --image_shape "512,512" \
#                         --out_folder {base_path}/vox_png_512px \
#                         --workers 32

In [ ]:
# # Zip and download if desired.
# # NOTE:
# # - This will be a massive amount of data (~1 TB) if you include all videos.
# # 
# from google.colab import files

# !zip -r /content/vox_pngs_512px.zip /content/video-preprocessing/vox_png_512px
# files.download("/content/vox_pngs_512px.zip")

In [6]:
# path_to_pngs = f'{base_path}/vox_png_512px'
# !ls -lh $path_to_pngs/train



---



---



## Run the following if video processing is interrupted before finishing.

In [56]:
import pandas as pd 
import numpy as np
import os
from pathlib import Path

In [57]:
base_path = Path.home()
print(base_path)
VIDEO_PREPROCESSING_REPO = f'{base_path}/video-preprocessing'
# !git clone https://github.com/jacobwjs/video-preprocessing $VIDEO_PREPROCESSING_REPO

%cd $VIDEO_PREPROCESSING_REPO

/home/jupyter
/home/jupyter/video-preprocessing


In [58]:
df = pd.read_csv(f'{VIDEO_PREPROCESSING_REPO}/vox-metadata.csv')
display(df.iloc[0:5]) # Display a few entries.

,video_id,start,end,bbox,fps,width,height,partition,person_id
0,WMh3ye5tBsE,3657,3994,624-48-1018-553,25,1280,720,train,id10005
1,WMh3ye5tBsE,3467,3576,610-41-989-550,25,1280,720,train,id10005
2,WMh3ye5tBsE,1426,1546,522-49-931-580,25,1280,720,train,id10005
3,WMh3ye5tBsE,3292,3435,600-59-1095-540,25,1280,720,train,id10005
4,WMh3ye5tBsE,4022,4235,534-42-1021-572,25,1280,720,train,id10005


In [71]:
# Get paths to already processed videos.
#
path_to_pngs = f'{base_path}/vox_png_512px'
test_dir = f'{path_to_pngs}/test'
train_dir = f'{path_to_pngs}/train'

# Get all processed videos in the train and test directories.
#
dirs = os.listdir(test_dir)
dirs += os.listdir(train_dir)

print("Processed: ", len(dirs))

Processed:  4402


4401


In [60]:
df_to_save = df.copy()
for d in dirs:
    person_id, video_id, start, end = d.split('#')
    start = int(start)
    end = int(end.split('.')[0])    
    
    # Find a match and drop the row.
    #
    idx = df.index[(df.person_id == person_id) & \
                   (df.video_id == video_id) & \
                   (df.start == start) & \
                   (df.end == end)]
    df_to_save.drop(idx, inplace=True)

In [65]:
print("Processed %d out of %d videos" % (len(df) - len(df_to_save), len(df)))

Processed 4455 out of 20075 videos


In [63]:
df_to_save.to_csv(f'{VIDEO_PREPROCESSING_REPO}/vox-metadata-512px-remaining.csv', index=False)

### Run the following in a terminal to continue converting videos to PNGs

In [ ]:
# python load_videos.py --metadata vox-metadata-512px-remaining.csv \
#     --format .png \
#     --image_shape "512,512" \t
#     --out_folder /home/jupyter/vox_png_512px \
#     --workers 16

### Check on how many videos have been processed.

In [76]:
from pathlib import Path
base_path = Path.home()

VIDEO_PREPROCESSING_REPO = f'{base_path}/video-preprocessing'
df = pd.read_csv(f'{VIDEO_PREPROCESSING_REPO}/vox-metadata.csv')

# Get paths to already processed videos.
#
path_to_pngs = f'{base_path}/vox_png_512px'
test_dir = f'{path_to_pngs}/test'
train_dir = f'{path_to_pngs}/train'

# Get all processed videos in the train and test directories.
#
dirs = os.listdir(test_dir)
dirs += os.listdir(train_dir)

print(path_to_pngs)
print("Processed: ", len(dirs))
print("Total: ", len(df))

/home/jupyter/vox_png_512px
Processed:  4441
Total:  20075
